In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import os, math, glob, re
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt

from kaggle_datasets import KaggleDatasets

from tensorflow.keras import layers
import tensorflow as tf

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

# Set Constants

In [ ]:
mri_types = ['FLAIR','T1w','T1wCE','T2w']
IMAGE_SIZE  = 128
IMAGE_DEPTH = 32
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
CHANNELS  = len(mri_types)

AUTO = tf.data.AUTOTUNE

In [ ]:
GCS_PATH = KaggleDatasets().get_gcs_path("rsna-brain-tumor-classification-tfrecords")
tf_train_path = GCS_PATH + "/tfrecords/train"
tf_valid_path = GCS_PATH + "/tfrecords/valid"

# Read TFRecords and Create Dataset

I have converted Original dataset with [this](https://www.kaggle.com/kavehshahhosseini/rsna-brain-tumor-convert-dicom-to-tfrecord) notebook to TFRecords and I have created a new dataset, which you can find it [here](https://www.kaggle.com/kavehshahhosseini/rsna-brain-tumor-classification-tfrecords). You should add the converted dataset, to the notebook to go further. I have splitted the data to train with 465 samples and each of them with shape `(128,128,32,4)` and validation data with 117 samples and the same shape.  

In [ ]:
def deserialize_example(serialized_string):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'MGMT_value': tf.io.FixedLenFeature([], tf.float32)
    }
    parsed_record = tf.io.parse_single_example(serialized_string, image_feature_description)
    image = tf.io.decode_raw(parsed_record['image'], tf.float64)
    image = tf.reshape(image,[IMAGE_SIZE,IMAGE_SIZE,IMAGE_DEPTH,CHANNELS])
    
    label = parsed_record['MGMT_value']
    return image, label

In [ ]:
train_set = tf.data.TFRecordDataset(str(tf_train_path + os.sep + "brain_train.tfrec"),
                                   compression_type="GZIP", num_parallel_reads=AUTO).map(deserialize_example).batch(BATCH_SIZE).prefetch(AUTO)
valid_set = tf.data.TFRecordDataset(str(tf_valid_path + os.sep + "brain_val.tfrec"),
                                   compression_type="GZIP", num_parallel_reads=AUTO).map(deserialize_example).batch(BATCH_SIZE).prefetch(AUTO)

# Visualize Image

In [ ]:
d = train_set.take(1)
for i, j in d:
    image = i
    label = j


img_id = np.random.randint(0, BATCH_SIZE)
channel = np.random.randint(0,CHANNELS)

plt.figure(figsize=(20,10),facecolor=(0,0,0))
cols = IMAGE_DEPTH//4
rows = 4

plt.axis("off")
for layer_idx in range(IMAGE_DEPTH):
    ax = plt.subplot(rows,cols,layer_idx+1)
    ax.imshow(np.squeeze(image[img_id,:,:,layer_idx,channel]), cmap="gray")
    ax.axis("off")
    ax.set_title(str(layer_idx+1),color='r',y=-0.01)
    
plt.suptitle(f"Batch Image NO.: {img_id}, MRI Type: {mri_types[channel]}, Shape: {image[img_id].shape}", color="w")
plt.subplots_adjust(wspace=0, hspace=0)
plt.show()

# Create Model

In [ ]:
def get_model(width=128, height=128, depth=32):

    act = "swish"
    ki = tf.keras.initializers.HeUniform(42)
    kr = 'l1_l2'
    inputs = tf.keras.Input((width, height, depth, 4))

    x = layers.Conv3D(filters=32, kernel_size=2, activation="swish", padding="same", kernel_regularizer=kr, kernel_initializer=ki)(inputs)
    x = layers.MaxPool3D(2)(x)

    x = layers.Conv3D(filters=64, kernel_size=2, activation="swish", padding="same", kernel_regularizer=kr, kernel_initializer=ki)(x)
    x = layers.MaxPool3D(2)(x)

    x = layers.Conv3D(filters=128, kernel_size=2, activation="swish", padding="same", kernel_regularizer=kr, kernel_initializer=ki)(x)
    x = layers.MaxPool3D(2)(x)
    
    x = layers.Conv3D(filters=256, kernel_size=2, activation="swish", padding="same", kernel_regularizer=kr, kernel_initializer=ki)(x)
    x = layers.MaxPool3D(2)(x)
    
    x = layers.Conv3D(filters=512, kernel_size=2, activation="swish", padding="same", kernel_regularizer=kr, kernel_initializer=ki)(x)
    x = layers.MaxPool3D(2)(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(units=128, activation="swish")(x)
    x = layers.Dense(units=128, activation="swish")(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    model = tf.keras.Model(inputs, outputs)

    return model


with strategy.scope():
    model = get_model(width=IMAGE_SIZE, height=IMAGE_SIZE, depth=IMAGE_DEPTH)
    model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy", "AUC"])

model.summary()

# Train Model

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_auc", patience=4, mode="max")
history = model.fit(train_set, validation_data=valid_set, epochs=20, callbacks=[early_stopping_cb])

# Plot Model Metrics

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

plt.figure(figsize=(16,7))
acc=history.history['auc']
val_acc=history.history['val_auc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1,len(acc)+1)
ax1 = plt.subplot(1,2,1)
ax1.plot(epochs, acc, 'r')
ax1.plot(epochs, val_acc, 'b')
ax1.set_xticks([i for i in epochs])
ax1.set_title('Training and validation AUC')
ax1.legend(["Training", "Validation" ])
ax1.set_xlabel("epochs")
ax1.set_ylabel("Accuracy")

ax2 = plt.subplot(1,2,2)
ax2.plot(epochs, loss, 'r')
ax2.plot(epochs, val_loss, 'b')
ax2.set_xticks([i for i in epochs])
ax2.legend(["Training", "Validation" ])
ax2.set_xlabel("Epochs")
ax2.set_ylabel("Loss")
ax2.set_title('Training and validation loss')

plt.show()

In [ ]:
model.save("my_simple_model_v3.h5")

# Prediction and Submission

I have created another kernel [here](https://www.kaggle.com/kavehshahhosseini/rsna-brain-tumor-tensorflow-tpu-tfrecord-inference) for prediction 